In [86]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler
pd.options.display.max_columns = 100

In [94]:
# Load the data into a pandas dataframe
train_data = pd.read_csv("data/fangraphs/batters/batters_19_21.csv")
test_data = pd.read_csv("data/fangraphs/batters/batters_22.csv")

In [95]:
# Runs Model

# Preprocess the data and select features for the model
# ... (fill in missing values, handle outliers, etc.)
train_data_chosen_cols = train_data[['G_x','HR_x', 'R_x',
       'SB_x', 'BB%_x', 'K%_x', 'ISO_x', 'BABIP', 'OBP_x', 'SLG_x',
       'wOBA_x', 'wRC+_x', 'BsR', 'Off', 
       'AB', 'H', '1B', '2B', '3B', 'BB', 'IBB', 'SO', 'HBP', 'CS',
       'PA', 'BB/K', 'AVG',
       'OPS', 'Spd', 'UBR',
       'wSB', 'wRC', 'wRAA',
       'BABIP', 'GB/FB', 'LD%', 'GB%', 'FB%', 'IFFB%', 'HR/FB', 'IFH', 'IFH%',
       'BUH', 'BUH%', 'Pull%', 'Cent%', 'Oppo%', 'Soft%', 'Med%', 'Hard%',
       #'Team',
       'O-Swing%', 'Z-Swing%', 'Swing%', 'O-Contact%',
       'Z-Contact%', 'Contact%', 'Zone%', 'F-Strike%', 'SwStr%', 'CStr%',
       'CSW%']]
train_data_chosen_cols = train_data_chosen_cols.rename(columns={'G_x':'G', 'HR_x':'HR', 'R_x':'R', 'SB_x':'SB', 'BB_x':'BB', 'SB_x':'SB', 'BB%_x':'BB%', 'K%_x':'K%', 'ISO_x':'ISO', 'OBP_x':'OBP', 'SLG_x':'SLG', 'wOBA_x': 'wOBA', 'wRC+_x':'wRC+'})

test_data_chosen_cols = test_data[['G_x','HR_x', 'R_x',
       'SB_x', 'BB%_x', 'K%_x', 'ISO_x', 'BABIP', 'OBP_x', 'SLG_x',
       'wOBA_x', 'wRC+_x', 'BsR', 'Off', 
       'AB', 'H', '1B', '2B', '3B', 'BB', 'IBB', 'SO', 'HBP', 'CS',
       'PA', 'BB/K', 'AVG',
       'OPS', 'Spd', 'UBR',
       'wSB', 'wRC', 'wRAA',
       'BABIP', 'GB/FB', 'LD%', 'GB%', 'FB%', 'IFFB%', 'HR/FB', 'IFH', 'IFH%',
       'BUH', 'BUH%', 'Pull%', 'Cent%', 'Oppo%', 'Soft%', 'Med%', 'Hard%',
       #'Team', 
       'O-Swing%', 'Z-Swing%', 'Swing%', 'O-Contact%',
       'Z-Contact%', 'Contact%', 'Zone%', 'F-Strike%', 'SwStr%', 'CStr%',
       'CSW%']]
test_data_chosen_cols = test_data_chosen_cols.rename(columns={'G_x':'G', 'HR_x':'HR', 'R_x':'R', 'SB_x':'SB', 'BB_x':'BB', 'SB_x':'SB', 'BB%_x':'BB%', 'K%_x':'K%', 'ISO_x':'ISO', 'OBP_x':'OBP', 'SLG_x':'SLG', 'wOBA_x': 'wOBA', 'wRC+_x':'wRC+'})


In [96]:
train_data_chosen_cols.columns

Index(['G', 'HR', 'R', 'SB', 'BB%', 'K%', 'ISO', 'BABIP', 'OBP', 'SLG', 'wOBA',
       'wRC+', 'BsR', 'Off', 'AB', 'H', '1B', '2B', '3B', 'BB', 'IBB', 'SO',
       'HBP', 'CS', 'PA', 'BB/K', 'AVG', 'OPS', 'Spd', 'UBR', 'wSB', 'wRC',
       'wRAA', 'BABIP', 'GB/FB', 'LD%', 'GB%', 'FB%', 'IFFB%', 'HR/FB', 'IFH',
       'IFH%', 'BUH', 'BUH%', 'Pull%', 'Cent%', 'Oppo%', 'Soft%', 'Med%',
       'Hard%', 'O-Swing%', 'Z-Swing%', 'Swing%', 'O-Contact%', 'Z-Contact%',
       'Contact%', 'Zone%', 'F-Strike%', 'SwStr%', 'CStr%', 'CSW%'],
      dtype='object')

In [97]:
train_data_chosen_cols.head(5)

,G,HR,R,SB,BB%,K%,ISO,BABIP,OBP,SLG,wOBA,wRC+,BsR,Off,AB,H,1B,2B,3B,BB,IBB,SO,HBP,CS,PA,BB/K,AVG,OPS,Spd,UBR,wSB,wRC,wRAA,BABIP,GB/FB,LD%,GB%,FB%,IFFB%,HR/FB,IFH,IFH%,BUH,BUH%,Pull%,Cent%,Oppo%,Soft%,Med%,Hard%,O-Swing%,Z-Swing%,Swing%,O-Contact%,Z-Contact%,Contact%,Zone%,F-Strike%,SwStr%,CStr%,CSW%
0,348,76,260,27,19.5%,16.8%,0.257,0.327,0.440,0.561,0.416,159,-4.3,110.7,1198,364,215,66,7,294,38,253,6,10,1509,1.16,0.304,1.001,4.7,-3.7,-0.2,312,125.6,0.327,1.47,19.9%,47.7%,32.4%,5.5%,24.7%,25,5.5%,1,20.0%,34.9%,39.4%,25.6%,14.3%,45.2%,40.5%,19.7%,65.2%,37.7%,63.4%,87.7%,80.1%,39.6%,53.9%,7.5%,18.1%,25.6%
1,329,59,249,79,7.2%,17.6%,0.213,0.355,0.370,0.530,0.379,135,14.0,81.1,1349,428,271,86,12,106,4,259,11,14,1474,0.41,0.317,0.900,7.1,8.2,8.7,259,76.3,0.355,1.37,20.6%,45.9%,33.5%,11.1%,16.0%,50,9.9%,1,100.0%,38.2%,35.9%,26.0%,16.8%,46.9%,36.2%,30.9%,71.4%,47.8%,63.1%,88.8%,79.1%,41.8%,64.6%,10.0%,16.8%,26.7%
2,273,81,210,52,10.1%,27.6%,0.303,0.347,0.369,0.596,0.399,153,13.7,94.6,1036,303,159,55,8,119,8,324,12,13,1175,0.37,0.292,0.965,6.6,7.9,4.0,226,80.8,0.347,1.22,19.9%,44.1%,36.1%,7.8%,31.4%,37,11.7%,2,40.0%,45.3%,33.6%,21.1%,15.3%,37.1%,47.6%,32.0%,76.0%,49.1%,50.3%,80.4%,68.4%,38.9%,60.5%,15.5%,12.4%,28.0%
3,339,76,224,61,10.8%,14.2%,0.261,0.262,0.350,0.528,0.365,131,14.9,70.5,1253,334,168,81,9,155,13,204,12,11,1432,0.76,0.267,0.877,6.4,5.5,6.5,234,57.4,0.262,0.72,19.7%,33.6%,46.7%,13.0%,15.4%,32,9.0%,2,66.7%,52.9%,28.5%,18.7%,18.4%,42.7%,38.9%,26.8%,65.1%,42.4%,77.4%,92.1%,86.6%,40.8%,53.4%,5.7%,17.9%,23.6%
4,223,70,174,14,17.4%,22.0%,0.336,0.320,0.431,0.631,0.431,174,10.0,102.4,786,232,113,44,5,172,23,217,21,3,987,0.79,0.295,1.062,4.8,4.7,0.8,218,94.6,0.320,0.57,25.6%,26.9%,47.5%,9.1%,25.5%,17,11.0%,0,0.0%,42.1%,34.0%,23.9%,12.3%,45.2%,42.5%,20.0%,61.2%,37.2%,67.7%,86.9%,80.9%,41.7%,55.2%,7.1%,19.7%,26.8%


In [98]:
cols_to_replace = ['BB%', 'K%', 'LD%', 'GB%', 'FB%', 'IFFB%', 'HR/FB', 'IFH%', 'BUH%', 'Pull%', 'Cent%', 'Oppo%', 'Soft%','Med%','Hard%', 'O-Swing%','Z-Swing%','Swing%','O-Contact%','Z-Contact%', 'Contact%','Zone%','F-Strike%','SwStr%','CStr%','CSW%']
train_data_chosen_cols[cols_to_replace] = train_data_chosen_cols[cols_to_replace].replace({'%':''}, regex=True)
test_data_chosen_cols[cols_to_replace] = test_data_chosen_cols[cols_to_replace].replace({'%':''}, regex=True)

In [99]:
train_data.head(5)

,Unnamed: 0,Name,Team_x,G_x,PA_x,HR_x,R_x,RBI_x,SB_x,BB%_x,K%_x,ISO_x,BABIP_x,AVG_x,OBP_x,SLG_x,wOBA_x,xwOBA,wRC+_x,BsR,Off,Def,WAR,playerid_x,Team_y,G_y,AB,PA_y,H,1B,2B,3B,HR_y,R_y,RBI_y,BB,IBB,SO,HBP,SF,SH,GDP,SB_y,CS,AVG_y,playerid_y,Team_x.1,PA,BB%_y,K%_y,BB/K,AVG,OBP_y,SLG_y,OPS,ISO_y,Spd,BABIP_y,UBR,wGDP,wSB,wRC,wRAA,wOBA_y,wRC+_y,playerid_x.1,Team_y.1,BABIP,GB/FB,LD%,GB%,FB%,IFFB%,HR/FB,IFH,IFH%,BUH,BUH%,Pull%,Cent%,Oppo%,Soft%,Med%,Hard%,playerid_y.1,Team,O-Swing%,Z-Swing%,Swing%,O-Contact%,Z-Contact%,Contact%,Zone%,F-Strike%,SwStr%,CStr%,CSW%,playerid
0,0,Juan Soto,WSN,348,1509,76,260,242,27,19.5%,16.8%,0.257,0.327,0.304,0.440,0.561,0.416,NaN,159,-4.3,110.7,-4.0,15.3,20123,WSN,348,1198,1509,364,215,66,7,76,260,242,294,38,253,6,11,0,35,27,10,0.304,20123,WSN,1509,19.5%,16.8%,1.16,0.304,0.440,0.561,1.001,0.257,4.7,0.327,-3.7,-0.4,-0.2,312,125.6,0.416,159,20123,WSN,0.327,1.47,19.9%,47.7%,32.4%,5.5%,24.7%,25,5.5%,1,20.0%,34.9%,39.4%,25.6%,14.3%,45.2%,40.5%,20123,WSN,19.7%,65.2%,37.7%,63.4%,87.7%,80.1%,39.6%,53.9%,7.5%,18.1%,25.6%,20123
1,1,Trea Turner,- - -,329,1474,59,249,175,79,7.2%,17.6%,0.213,0.355,0.317,0.370,0.530,0.379,NaN,135,14.0,81.1,11.0,13.7,16252,- - -,329,1349,1474,428,271,86,12,59,249,175,106,4,259,11,8,0,33,79,14,0.317,16252,- - -,1474,7.2%,17.6%,0.41,0.317,0.370,0.530,0.900,0.213,7.1,0.355,8.2,-2.9,8.7,259,76.3,0.379,135,16252,- - -,0.355,1.37,20.6%,45.9%,33.5%,11.1%,16.0%,50,9.9%,1,100.0%,38.2%,35.9%,26.0%,16.8%,46.9%,36.2%,16252,- - -,30.9%,71.4%,47.8%,63.1%,88.8%,79.1%,41.8%,64.6%,10.0%,16.8%,26.7%,16252
2,2,Fernando Tatis Jr.,SDP,273,1175,81,210,195,52,10.1%,27.6%,0.303,0.347,0.292,0.369,0.596,0.399,NaN,153,13.7,94.6,3.8,13.6,19709,SDP,273,1036,1175,303,159,55,8,81,210,195,119,8,324,12,8,0,15,52,13,0.292,19709,SDP,1175,10.1%,27.6%,0.37,0.292,0.369,0.596,0.965,0.303,6.6,0.347,7.9,1.8,4.0,226,80.8,0.399,153,19709,SDP,0.347,1.22,19.9%,44.1%,36.1%,7.8%,31.4%,37,11.7%,2,40.0%,45.3%,33.6%,21.1%,15.3%,37.1%,47.6%,19709,SDP,32.0%,76.0%,49.1%,50.3%,80.4%,68.4%,38.9%,60.5%,15.5%,12.4%,28.0%,19709
3,3,Jose Ramirez,CLE,339,1432,76,224,232,61,10.8%,14.2%,0.261,0.262,0.267,0.350,0.528,0.365,NaN,131,14.9,70.5,10.9,13.2,13510,CLE,339,1253,1432,334,168,81,9,76,224,232,155,13,204,12,12,0,23,61,11,0.267,13510,CLE,1432,10.8%,14.2%,0.76,0.267,0.350,0.528,0.877,0.261,6.4,0.262,5.5,2.9,6.5,234,57.4,0.365,131,13510,CLE,0.262,0.72,19.7%,33.6%,46.7%,13.0%,15.4%,32,9.0%,2,66.7%,52.9%,28.5%,18.7%,18.4%,42.7%,38.9%,13510,CLE,26.8%,65.1%,42.4%,77.4%,92.1%,86.6%,40.8%,53.4%,5.7%,17.9%,23.6%,13510
4,4,Mike Trout,LAA,223,987,70,174,168,14,17.4%,22.0%,0.336,0.320,0.295,0.431,0.631,0.431,NaN,174,10.0,102.4,-4.3,13.1,10155,LAA,223,786,987,232,113,44,5,70,174,168,172,23,217,21,8,0,6,14,3,0.295,10155,LAA,987,17.4%,22.0%,0.79,0.295,0.431,0.631,1.062,0.336,4.8,0.320,4.7,4.5,0.8,218,94.6,0.431,174,10155,LAA,0.320,0.57,25.6%,26.9%,47.5%,9.1%,25.5%,17,11.0%,0,0.0%,42.1%,34.0%,23.9%,12.3%,45.2%,42.5%,10155,LAA,20.0%,61.2%,37.2%,67.7%,86.9%,80.9%,41.7%,55.2%,7.1%,19.7%,26.8%,10155


In [103]:
# Split the data into training and testing sets
train_data_x = train_data_chosen_cols.drop('R', axis=1)
train_data_y = train_data_chosen_cols['R']
test_data_x = test_data_chosen_cols.drop('R', axis=1)
test_data_y = test_data_chosen_cols['R']

# Scale x variable datasets
scaler = MinMaxScaler()
train_data_x = scaler.fit_transform(train_data_x)
test_data_x = scaler.fit_transform(test_data_x)

# Train a random forest regressor
rf = RandomForestRegressor(n_estimators=100)
rf.fit(train_data_x, train_data_y)

# Make predictions on the test set
predictions = rf.predict(test_data_x)

# Assign the predictions to the players in the test set
test_data_y_compare = pd.DataFrame({'Actual_Runs': test_data_y, 'Predicted_Runs': predictions})

# Add name back to test_data_y
test_data_y_compare["Name"] = test_data["Name"]

# Sort the players by their predicted fantasy value
sorted_data = test_data_y_compare.sort_values("Predicted_Runs", ascending=False)

# Display the top 10 players with the highest predicted fantasy value
print(test_data_y_compare[["Name", "Predicted_Runs", "Actual_Runs"]].head(10))


               Name  Predicted_Runs  Actual_Runs
0       Aaron Judge          249.25          133
1     Manny Machado          196.17          100
2     Nolan Arenado          195.45           73
3  Paul Goldschmidt          231.33          106
4   Freddie Freeman          228.32          117
5  Francisco Lindor          183.65           98
6    Yordan Alvarez          189.89           95
7      Mookie Betts          197.18          117
8       Jose Altuve          195.92          103
9     J.T. Realmuto          157.70           75


In [ ]:
test_data_y_compare

,Actual_Runs,Predicted_Runs
0,133,244.97
1,100,197.51
2,73,194.20
3,106,226.38
4,117,225.90
...,...,...
312,36,68.43
313,43,83.13
314,27,39.03
315,29,70.56
